# 1. Context

This Notebook evaluates Character Error Rate (CER) & Word Error Rate (WER) in OCRed Document at paragraph compared to Ground Truth

> Accuracy evaluation will be scoped to document (1 page) and multi column layput in later studies

# 2. Imports

In [1]:
import jiwer

In [2]:

# The correct, ground truth text
reference_text = "मीना के परिवार में सात लोग हैं - उसके दादा, दादी, माता, पिता, चाचा, मीना और उसका छोटा भाई दिवाकर। दिवाकर तीन साल का है। वह बहुत नटखट और चुलबुला है।"
# The OCR output with errors
hypothesis_text = "मीना के परिवार में सात लाग हैं - उसके दादा, दाडी, माता, पिता, चाच, मीना और उसका छोटा भाई दिवाकर। दिवादर तीन साल का है। वह बहुर नटखट और चुलबुला है।"

output = jiwer.process_characters(reference_text, hypothesis_text)

In [3]:
print(jiwer.visualize_alignment(output))

=== SENTENCE 1 ===

REF: मीना के परिवार में सात लोग हैं - उसके दादा, दादी, माता, पिता, चाचा, मीना और उसका छोटा भाई दिवाकर। दिवाकर तीन साल का है। वह बहुत नटखट और चुलबुला है।
HYP: मीना के परिवार में सात लाग हैं - उसके दादा, दाडी, माता, पिता, चाच*, मीना और उसका छोटा भाई दिवाकर। दिवादर तीन साल का है। वह बहुर नटखट और चुलबुला है।
                             S                     S                  D                                    S                       S                    

=== SUMMARY ===
number of sentences: 1
substitutions=4 deletions=1 insertions=0 hits=142

cer=3.40%



In [4]:
print(jiwer.visualize_error_counts(output))

=== SUBSTITUTIONS ===
ो --> ा = 1x
द --> ड = 1x
क --> द = 1x
त --> र = 1x

=== INSERTIONS ===
none
=== DELETIONS ===
ा = 1x


In [5]:
def visualize_character_errors(
    reference: str,
    hypothesis: str,
    chunks
):
    """
    Draws the reference and hypothesis texts with highlighted errors.

    Args:
        reference (str): The ground truth text.
        hypothesis (str): The transcribed text.
        chunks (List[AlignmentChunk]): A list of AlignmentChunk objects from jiwer.
    """
    # ANSI escape codes for colors
    GREEN_BG = "\033[42m"
    RED_BG = "\033[41m"
    RESET = "\033[0m"

    ref_viz = ""
    hyp_viz = ""

    for chunk in chunks:
        ref_substring = reference[chunk.ref_start_idx:chunk.ref_end_idx]
        hyp_substring = hypothesis[chunk.hyp_start_idx:chunk.hyp_end_idx]

        if chunk.type == 'equal':
            ref_viz += ref_substring
            hyp_viz += hyp_substring
        elif chunk.type == 'substitute':
            # Highlight substitutions with different background colors
            ref_viz += f"{RED_BG}{ref_substring}{RESET}"
            hyp_viz += f"{GREEN_BG}{hyp_substring}{RESET}"
        elif chunk.type == 'delete':
            # Highlight deletions in the reference and add placeholders in hypothesis
            ref_viz += f"{RED_BG}{ref_substring}{RESET}"
            hyp_viz += "*" * len(ref_substring)
        elif chunk.type == 'insert':
            # Highlight insertions in the hypothesis and add placeholders in reference
            ref_viz += "*" * len(hyp_substring)
            hyp_viz += f"{GREEN_BG}{hyp_substring}{RESET}"

    print(f"Reference:\n{ref_viz}\n")
    print(f"Hypothesis:\n{hyp_viz}\n")


In [6]:
visualize_character_errors(reference_text, hypothesis_text, output.alignments[0])

Reference:
मीना के परिवार में सात लोग हैं - उसके दादा, दादी, माता, पिता, चाचा, मीना और उसका छोटा भाई दिवाकर। दिवाकर तीन साल का है। वह बहुत नटखट और चुलबुला है।

Hypothesis:
मीना के परिवार में सात लाग हैं - उसके दादा, दाडी, माता, पिता, चाच*, मीना और उसका छोटा भाई दिवाकर। दिवादर तीन साल का है। वह बहुर नटखट और चुलबुला है।

